<a href="https://colab.research.google.com/github/renanalencar/projeto-md-emocoes/blob/main/entregas/Assign9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import numpy as np

# 1. Dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# importar os pacotes necessários
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
arquivo_tedio = "/content/drive/Shareddrives/MINERAÇÃO DE DADOS ENG74291/Assign6/dadosTedioProcessados.xlsx"
df_tedio = pd.read_excel(arquivo_tedio, index_col=0)

# CSV apenas com as emoções. Pontos euclidianos foram removidos
# deixar no dataframe somente com as linhas correspondentes ao game_id 1
df_tedio = df_tedio.drop(df_tedio.iloc[:, 9:35], axis=1)
filtro  = df_tedio["game_id"] == 1
df_tedio = df_tedio[filtro]

# remove a coluna game_name
#del df_tedio["game_name"]

# visualizar as 8 primeiras entradas do df
df_tedio.head(8)

In [ ]:
# verificar o tamanho do df
print("Variáveis:\t {}".format(df_tedio.shape[1]))
print("Entradas:\t {}".format(df_tedio.shape[0]))

In [ ]:
# vizualisar os nomes das colunas
df_tedio.columns

In [ ]:
# descobrir os tipos das variáveis
df_tedio.dtypes

## 1.2 Criando classes para a base de dados

In [ ]:
# função para classificar os dados como tédio (1) ou estresse (0)
def def_tedio(c):
  if c['angry'] >= 0.1 and c['disgusted'] >= 0.1:
    return 1
  elif c['sad'] >= 0.1 and c['surprised'] >= 0.1:
    return 1
  else:
    return 0

In [ ]:
# criar a coluna 'target' para o df_tedio
df_tratado = pd.DataFrame(df_tedio)

df_tratado['target'] = df_tedio.apply(def_tedio, axis=1)

df_tratado.head(8)

In [ ]:
# verificar o tipo de dado de cada coluna
df_tratado.dtypes

In [ ]:
# verificar que colunas tem valores NaN (Not a Number)
df_tratado.isnull().any()

In [ ]:
# Completando os registros que tem valor NaN com 0.0
df_tratado = df_tratado.fillna(0.0)

## 1.3 Conjunto de dados para treinamento e teste

In [ ]:
# criar os conjuntos de dados e classes para treinamento e teste
from sklearn.model_selection import train_test_split

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(df_tratado.drop(columns=['target']), df_tratado.target, test_size=0.3)

## 1.4 Correlação
Baseado em [Como selecionar as melhores features para seu modelo de Machine Learning](https://paulovasconcellos.com.br/como-selecionar-as-melhores-features-para-seu-modelo-de-machine-learning-2e9df83d062a)

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 7))
sns.heatmap(df_tratado.corr(),
            annot = True,
            fmt = '.2f',
            cmap='Blues')
plt.title('Correlação entre variáveis do dataset de tédio')
plt.show()

O valor mostrado para cada correlação vai de -1 — que indica uma correlação negativa perfeita — a +1 — uma correlação positiva perfeita. Vale lembrar que a função .corr() traz, por padrão, a correlação de Pearson, mostrando um relacionamento linear entre as variáveis. Em casos onde há um relacionamento não-linear, a matriz pode não ser uma boa medida.

## 1.5 Feature Importance
O feature_importance_ retornar um array onde cada elemento dele é uma feature do seu modelo. Ele irá dizer, em proporções, quão importante aquela feature é para o modelo, onde quanto maior o valor, mais importante a feature é para o modelo.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_RFC  = RandomForestClassifier()
clf_RFC.fit(X_treinamento, y_treinamento)

# Mostrando importância de cada feature
clf_RFC.feature_importances_

É retornado um array com quatro elementos. Se você somar todos eles, verá que o resultado será 1. Ao analisar esse array, podemos ver que a feature mais importante para o algoritmo Floresta Randômica foi a variável 'surprised'.

In [ ]:
importances = pd.Series(data=clf_RFC.feature_importances_, index=df_tratado.columns[0:9])
sns.barplot(x=importances, y=importances.index, orient='h').set_title('Importância de cada feature')

As vezes, os valores mostrados pelo feature_importances_ pode ser enviesado dependendo dos parâmetros definidos na criação do objeto. Evitar usar os parâmetros default do Floresta Randômica.

# 2. Regressões

## 2.1 Regressão Linear Simples e Multipla


*   [Linear Regression Example](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html)
*   [Implementando Regressão Linear Simples em Python](https://medium.com/data-hackers/implementando-regress%C3%A3o-linear-simples-em-python-91df53b920a8)
*   [Regressão Linear](https://www.kaggle.com/marilivb/4-regress-o-linear)
*   [https://www.datageeks.com.br/regressao-linear/](https://www.datageeks.com.br/regressao-linear/)

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
# Criar um objeto de regressão linear
regr = linear_model.LinearRegression()

# Treinar o modelo usando os conjuntos de treinamento
regr.fit(X_treinamento, y_treinamento)

# Fazer predições usando o conjunto de teste
y_pred = regr.predict(X_teste)

In [ ]:
# Os coeficientes encontrados
print('Coeficientes: \n', regr.coef_)
# O Erro Médio Quadrático (EMQ)
print('Erro Médio Quadrático (EMQ): %.2f'
      % mean_squared_error(y_teste, y_pred))
# O coeficiente de determinação: 1 é a predição perfeita
print('Coeficiente de determinação: %.2f'
      % r2_score(y_teste, y_pred))

### 2.1.1 Diagrama de Dispersão

In [ ]:
# plotar saídas
plt.scatter(X_teste.iloc[:,0].values, y_teste,  color='black')
plt.plot(X_teste, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

## 2.2 Regressão Logística


*   [Logistic Regression using Python (scikit-learn)](https://towardsdatascience.com/logistic-regression-using-python-sklearn-numpy-mnist-handwriting-recognition-matplotlib-a6b31e2b166a)
*   [Regressão Logística e Métricas de Classificação em Python](http://neylsoncrepalde.github.io/2019-11-25-regressao_logistica_python/)



In [ ]:
from sklearn.linear_model import LogisticRegression

clf_LR = LogisticRegression(random_state=0).fit(X_treinamento, y_treinamento)
clf_LR.predict(X_teste)

In [ ]:
clf_LR.predict_proba(X_teste)

In [ ]:
clf_LR.score(X_treinamento, y_treinamento)

### 2.2.1 Diagrama de Dispersão

In [ ]:
# plotar saídas
plt.scatter(X_teste.iloc[:,0].values, y_teste,  color='black')
plt.plot(X_teste, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

# 3. Detecção de anomalias
Referências:


*   [Anomaly Detection Techniques in Python](https://medium.com/learningdatascience/anomaly-detection-techniques-in-python-50f650c75aaf)
*   [4 Automatic Outlier Detection Algorithms in Python](https://machinelearningmastery.com/model-based-outlier-detection-and-removal-in-python/)
*   [Learn how to develop highly accurate models to detect anomalies using Artificial Neural Networks with the Tensorflow library in Python3.](https://outline.com/D8jZMf)


## 3.1 Métodos Estatísticos

### 3.1.1 Paramétricos: Diagrama de Caixa

In [ ]:
boxplot = df_tratado.boxplot(column=['angry', 'disgusted', 'fear', 'sad', 'surprised', 'happy'])

In [ ]:
df_emocoes = df_tratado.copy(deep=True)
df_emocoes = df_emocoes.drop(columns=['game_id', 'uuid', 'timestamp', 'target'])
ax = sns.violinplot(data=df_emocoes)

### 3.1.2 Não Paramétricos: Análise de Histograma

In [ ]:
#ax = df_emocoes.plot.hist(bins=12, alpha=0.5)
ax = df_emocoes.plot.hist()

## 3.2 Métodos Algorítmicos

### 3.2.1 Proximidade: Local Outlier Factor (LOF)
Referências:

*   [2.7. Novelty and Outlier Detection](https://scikit-learn.org/stable/modules/outlier_detection.html)
*   [sklearn.neighbors.LocalOutlierFactor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.LocalOutlierFactor.html)
*   [Outlier detection with Local Outlier Factor (LOF)](https://scikit-learn.org/stable/auto_examples/neighbors/plot_lof_outlier_detection.html)
*   [Anomaly detection with Local Outlier Factor (LOF)](https://towardsdatascience.com/anomaly-detection-with-local-outlier-factor-lof-d91e41df10f2)

### 3.2.2 Redes Neurais: Redes Neurais Supervisionadas
Referências:


*   [Comparing anomaly detection algorithms for outlier detection on toy datasets](https://scikit-learn.org/0.20/auto_examples/plot_anomaly_comparison.html)
*   [How to use machine learning for anomaly detection and condition monitoring](https://towardsdatascience.com/how-to-use-machine-learning-for-anomaly-detection-and-condition-monitoring-6742f82900d7)



## 3.3 Machine Learning

### 3.3.1. Árvores de decisão

### 3.3.2 Florestas de isolamento (*Isolation Forest*)
Referências:

*   [2.7. Novelty and Outlier Detection](https://scikit-learn.org/stable/modules/outlier_detection.html)
*   [sklearn.ensemble.IsolationForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html)
*   [Anomaly Detection with Isolation Forest & Visualization](https://towardsdatascience.com/anomaly-detection-with-isolation-forest-visualization-23cd75c281e2)
*   [Anomaly Detection Using Isolation Forest in Python](https://blog.paperspace.com/anomaly-detection-isolation-forest/)


In [ ]:
from sklearn.ensemble import IsolationForest
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(df_emocoes[['angry']])

In [ ]:
df = pd.DataFrame(df_emocoes['angry'])
df['scores']=model.decision_function(df[['angry']])
df['anomaly']=model.predict(df[['angry']])
df.head(20)

In [ ]:
anomaly=df.loc[df['anomaly']==-1]
anomaly_index=list(anomaly.index)
print(anomaly)

In [ ]:
outliers_counter = len(df[df['angry'] > 0.5])
outliers_counter

In [ ]:
print("Accuracy percentage:", 100*list(df['anomaly']).count(-1)/(outliers_counter))

In [ ]:
#specify the 12 metrics column names to be modelled
to_model_columns=df_emocoes[0:7]
to_model_columns

clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
clf.fit(df_emocoes)
pred = clf.predict(df_emocoes)
df_emocoes['anomaly']=pred
outliers=df_emocoes.loc[df_emocoes['anomaly']==-1]
outlier_index=list(outliers.index)
print(outlier_index)
#Find the number of anomalies and normal points here points classified -1 are anomalous
print(df_emocoes['anomaly'].value_counts())

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(2)
pca.fit(df_emocoes)
res=pd.DataFrame(pca.transform(df_emocoes))
Z = np.array(res)
plt.title("IsolationForest")
plt.contourf( Z, cmap=plt.cm.Blues_r)
b1 = plt.scatter(res[0], res[1], c='green', s=20,label="normal points")
b1 =plt.scatter(res.iloc[outlier_index,0],res.iloc[outlier_index,1], c='green',s=20,  edgecolor="red",label="predicted outliers")
plt.legend(loc="upper right")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
pca = PCA(n_components=3)  # Reduce to k=3 dimensions
scaler = StandardScaler()
#normalize the metrics
X = scaler.fit_transform(df_emocoes)
X_reduce = pca.fit_transform(X)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_zlabel("x_composite_3")# Plot the compressed data points
ax.scatter(X_reduce[:, 0], X_reduce[:, 1], zs=X_reduce[:, 2], s=4, lw=1, label="inliers",c="green")# Plot x's for the ground truth outliers
ax.scatter(X_reduce[outlier_index,0],X_reduce[outlier_index,1], X_reduce[outlier_index,2],
           lw=2, s=60, marker="x", c="red", label="outliers")
ax.legend()
plt.show()